In [169]:
import pandas as pd
import re
from bs4 import BeautifulSoup 
import requests
import time

In [ ]:
#Get the year's HTML page
page = requests.get("https://www.necta.go.tz/results/2018/acsee/acsee.htm").text
soup = BeautifulSoup(page, "lxml")

In [ ]:
#Put all the links in a list
schoolUrls2018 = ["https://www.necta.go.tz/results/2018/acsee/"+i['href'] for i in soup.find_all("table")[2].find_all("a")]

In [ ]:
#Some helper functions

def list_subjects(cell):
    subject_list = [subject.replace("'", '') for subject in re.split("'\s", cell)]
    #subject_list.sort() #thought I may want to sort the list, but I don't think that step is necessary.
    return subject_list

def pull_subject(subj):
    return re.search(r'(^[A-Za-z\/]*)', subj).group()

def make_subject_columns(df):
    #Slight inspired by this: https://stackoverflow.com/questions/44663903/pandas-split-column-of-lists-of-unequal-length-into-multiple-columns
    for row in df.itertuples():
        for subject in row[5]:
            df.loc[row[0], pull_subject(subject)] = subject
    return df

def clean_grades(df):
    df.fillna("X", inplace=True)
    df.iloc[:, 5:] = df.iloc[:, 5:].applymap(lambda x: x[-1])
    del df["DETAILED SUBJECTS"]
    return df

#Put all the tables in a list, then concat the tables into a single dataframe
all_acsee = []
for i in schoolUrls2018:
    try:
        alevel = pd.read_html(i, header=0)[0]

        #Listify the list of subject grades, grades for each row in Detailed Subjects column
        alevel.loc[:,"DETAILED SUBJECTS"] = alevel.loc[:,"DETAILED SUBJECTS"].apply(lambda x: list_subjects(x))

        #Split the grades and make columns for them
        make_subject_columns(alevel)

        #Tidy up grade data
        clean_grades(alevel)

        #Add this school's table to the list of all_acsee schools
        all_acsee.append(alevel)
        time.sleep(10)
    except Exception as e:
        print((schoolUrls2018.index(i), i, e))
        time.sleep(60)
        continue

all_acsee_df = pd.concat(all_acsee, axis=0, ignore_index=True)
all_acsee_df.head()

#F column is: all_acsee_fullDF.rename(columns={'F': 'F & HN NUTRITION'}, inplace=True)
#Empty string column, '', is *W *E *R students who don't have any subject data. del all_acsee_fullDF['']
#Re-order the columns by calling .columns and copy-pasting into a subselect in preferred order

In [205]:
all_acsee_fullDF = all_acsee_fullDF[['CNO', 'SEX', 'AGGT','DIV', 'ACCOUNTANCY', 'ADV/MATHS', 'AGRICULTURE', 'ARABIC', 'BAM',
       'BIOLOGY', 'CHEMISTRY', 'COMMERCE', 'COMP/SCIENCE',
       'DIVINITY', 'ECONOMICS', 'ENGLISH', 'F & HN NUTRITION', 'FRENCH',
       'G/STUDIES', 'GEOGR', 'HISTORY', 'IS/KNOWLEDGE', 'KISWAHILI', 'PHYSICS']]
all_acsee_fullDF.to_csv("./CompleteDatasets/necta_acsee_2018.csv")

In [144]:
#make long form
alvl.melt(id_vars=["CNO", "SEX", "AGGT", "DIV"], var_name="Subjects", value_name="Grade").sort_values("CNO")

,CNO,SEX,AGGT,DIV,Subjects,Grade
0,P0133/0501,F,13,III,G/STUDIES,F
32,P0133/0501,F,13,III,GEOGR,X
48,P0133/0501,F,13,III,CHEMISTRY,X
56,P0133/0501,F,13,III,BIOLOGY,X
24,P0133/0501,F,13,III,ENGLISH,D
64,P0133/0501,F,13,III,BAM,X
16,P0133/0501,F,13,III,KISWAHILI,C
72,P0133/0501,F,13,III,ADV/MATHS,X
40,P0133/0501,F,13,III,PHYSICS,X
8,P0133/0501,F,13,III,HISTORY,C
